In [1]:
import os
import sys
project_root = os.path.abspath('../src')
if project_root not in sys.path:
    sys.path.insert(0, project_root)

import wandb
key = None
if key is not None:
    wandb.login(key=key)
    os.environ['WANDB_API_KEY'] = key
os.environ["WANDB_NOTEBOOK_NAME"] = "uci_har.ipynb"

In [2]:
!nvidia-smi

Fri May 23 10:28:41 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.127.08             Driver Version: 550.127.08     CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla V100-SXM2-32GB           Off |   00000000:06:00.0 Off |                    0 |
| N/A   32C    P0             41W /  300W |       1MiB /  32768MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

In [4]:
import pandas as pd

# Load training data
fol = "data/uci_har/"
X_train = pd.read_csv(fol + 'UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
y_train = pd.read_csv(fol + 'UCI HAR Dataset/train/y_train.txt', header=None, names=['activity'])
subject_train = pd.read_csv(fol + 'UCI HAR Dataset/train/subject_train.txt', header=None, names=['subject'])
y_train -= 1
# Load test data
X_test = pd.read_csv(fol + 'UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
y_test = pd.read_csv(fol + 'UCI HAR Dataset/test/y_test.txt', header=None, names=['activity'])
subject_test = pd.read_csv(fol + 'UCI HAR Dataset/test/subject_test.txt', header=None, names=['subject'])
y_test -= 1
features = pd.read_csv(fol + 'UCI HAR Dataset/features.txt', header=None, delim_whitespace=True)
X_train.columns = features[1].values
X_test.columns = features[1].values

/tmp/ipykernel_151599/213457286.py:5: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv(fol + 'UCI HAR Dataset/train/X_train.txt', delim_whitespace=True, header=None)
/tmp/ipykernel_151599/213457286.py:10: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv(fol + 'UCI HAR Dataset/test/X_test.txt', delim_whitespace=True, header=None)
/tmp/ipykernel_151599/213457286.py:14: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  features = pd.read_csv(fol + 'UCI HAR Dataset/features.txt', header=None, delim_whitespace=True)


In [5]:
print(subject_train.shape)

(7352, 1)


In [6]:
import numpy as np
print(X_train.shape, y_train.shape, subject_train.shape)
print(X_test.shape, y_test.shape, subject_test.shape)
print(np.unique(y_train.to_numpy().flatten()))
reverse_mp = {0: 'WALKING', 1: 'WALKING_UPSTAIRS', 2: 'WALKING_DOWNSTAIRS', 3: 'SITTING', 4: 'STANDING', 5: 'LAYING'}
mp = {y: i for i, y in reverse_mp.items()}

(7352, 561) (7352, 1) (7352, 1)
(2947, 561) (2947, 1) (2947, 1)
[0 1 2 3 4 5]


In [7]:
# Example data: X (features), y (labels), groups (subject IDs)
from sklearn.model_selection import GroupShuffleSplit
splitter = GroupShuffleSplit(test_size=0.1, n_splits=1, random_state=42)
train_idx, val_idx = next(splitter.split(X_train, y_train, groups=subject_train))
print(train_idx.shape, val_idx.shape)
X_val = X_train.iloc[val_idx, :]
X_train = X_train.iloc[train_idx, :]
y_val = y_train.iloc[val_idx]
y_train = y_train.iloc[train_idx]

(6220,) (1132,)


In [8]:
print(X_train.shape, y_train.shape, X_val.shape, y_val.shape, X_test.shape, y_test.shape)

(6220, 561) (6220, 1) (1132, 561) (1132, 1) (2947, 561) (2947, 1)


In [9]:
import torch
def normalize_data(X):
    X_mean = X.mean(axis=0)
    X_std = X.std(axis=0)
    return (X - X_mean) / X_std, X_mean, X_std

X_train, X_train_mean, X_train_std = normalize_data(X_train)
print(X_train_mean.shape, X_train_std.shape)
X_test = (X_test - X_train_mean) / X_train_std
X_val = (X_val - X_train_mean) / X_train_std

train_set = torch.utils.data.TensorDataset(torch.from_numpy(X_train.to_numpy()).float(), torch.from_numpy(y_train.to_numpy().flatten()).long())
test_set = torch.utils.data.TensorDataset(torch.from_numpy(X_test.to_numpy()).float(), torch.from_numpy(y_test.to_numpy().flatten()).long())
val_set = torch.utils.data.TensorDataset(torch.from_numpy(X_val.to_numpy()).float(), torch.from_numpy(y_val.to_numpy().flatten()).long())



(561,) (561,)


In [10]:
print(len(train_set), len(test_set), len(val_set))

6220 2947 1132


In [11]:
from run_experiment import run_expriment
import random
import string

#Comment this lines for relu


# activation = 'relu'
# enable_boundary_loss = False
# disable_batchnorm_grad_clip_exclusion = True
# lambda_penalty = -1
# gradient_clip_val = None



activation = 'poly'
enable_boundary_loss = True
disable_batchnorm_grad_clip_exclusion = False
lambda_penalty = 10
gradient_clip_val = 1.0


pol_degree = 2

pol_degree_map = {
    2:{"B": 13, "penalty_B": 13},
    4:{"B": 20, "penalty_B": 20 * 0.75},
    8:{"B": 35, "penalty_B": 30 * 0.5},
}

max_epoch = 10
dropout = 0.0
learning_rate = 0.001


project_name = "test"
data_workers = 4
model = "mlp"
dataset = {
    "train": train_set,
    "val": val_set,
    "test": test_set
}

run_id = "test_mlp"
custom_tag = "test"

ori_activaiton = "relu"
samp_size = 100

B = pol_degree_map[pol_degree]["B"]
penalty_B = pol_degree_map[pol_degree]["penalty_B"]
boundary_loss_params = {'type': 'exp', 'penalty_B':  penalty_B, 'acc_norm': 'sum'}
learnable_coeffs = True
input_size = X_train.shape[1]


optimizer_params = {
        'type': 'adamw',
        'lr': learning_rate,
        'params': {
        }
}
scheduler_params = {'type': 'reduce_on_plateau',
                        'params': {
                                'mode': 'min',
                                'factor': 0.1,
                                'patience': 5,
                                'threshold': 0.1,
                                'verbose': True
                        },
                        'monitor': 'val_acc_epoch'
}

boundary_loss_params = {'type': 'exp', 'penalty_B': B * 1.0, 'acc_norm': 'sum'}
actvation_params =  {
        "ori_activation": ori_activaiton,
        'B': B,
        'samp_size': samp_size,
        'pol_degree': pol_degree,
        'learnable_coeffs': learnable_coeffs,
        'initialization': "least_square",
        'boundary_loss_params': boundary_loss_params

    }
num_classes = np.unique(y_train).shape[0]
model_params = {
    "use_singleton_activation": False,
    "bn_before_act": False,
    "activation": activation,
    "dropout": dropout,
    "num_classes":num_classes,
    "actvation_params": actvation_params,
    "model":model,
    "input_size": input_size,
    'hidden_dims': [256, 128, 64, 32, 16]
}

training_params = {
    "enable_boundary_loss": enable_boundary_loss,
    "gradient_clip_val": gradient_clip_val,
    "max_epoch": max_epoch,
    "lambda_penalty": lambda_penalty,
    "disable_batchnorm_grad_clip_exclusion":disable_batchnorm_grad_clip_exclusion,
    'optimizer_params': optimizer_params,
    'scheduler_params': scheduler_params

}

dataset_params = {
    "data_workers": data_workers,
    "dataset": dataset,
    "batch_size": 800
}

project_params = {"run_id": run_id,
                  "project_name": project_name,
                  "custom_tag": custom_tag
                  }

run_expriment(project_params=project_params, dataset_params=dataset_params,
               model_params=model_params, training_params=training_params)


B 13 samp_size 100 initialization least_square pol_degree 2
B 13 samp_size 100 initialization least_square pol_degree 2
B 13 samp_size 100 initialization least_square pol_degree 2
B 13 samp_size 100 initialization least_square pol_degree 2
B 13 samp_size 100 initialization least_square pol_degree 2
B 13 samp_size 100 initialization least_square pol_degree 2


GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
HPU available: False, using: 0 HPUs
/opt/conda/envs/hfe/lib/python3.12/site-packages/pytorch_lightning/trainer/connectors/logger_connector/logger_connector.py:75: Starting from v1.9.0, `tensorboardX` has been removed as a dependency of the `pytorch_lightning` package, due to potential conflicts with other packages in the ML ecosystem. For this reason, `logger=True` will use `CSVLogger` as the default logger, unless the `tensorboard` or `tensorboardX` packages are found. Please `pip install lightning[extra]` or one of them to enable TensorBoard support by default
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
/opt/conda/envs/hfe/lib/python3.12/site-packages/torch/optim/lr_scheduler.py:28: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn("The verbose parameter is deprecated. Please use get_last_lr() "

  | Name           | Type             | Pa

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/opt/conda/envs/hfe/lib/python3.12/site-packages/pytorch_lightning/loops/fit_loop.py:298: The number of training batches (8) is smaller than the logging interval Trainer(log_every_n_steps=50). Set a lower value for log_every_n_steps if you want to see logs for the training epoch.


Training: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Restoring states from the checkpoint path at /code/Polynomial-NN/notebooks/lightning_logs/version_60/checkpoints/best-acc-epoch=8-val_acc=0.00.ckpt
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loaded model weights from the checkpoint at /code/Polynomial-NN/notebooks/lightning_logs/version_60/checkpoints/best-acc-epoch=8-val_acc=0.00.ckpt


────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Validate metric               DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val_acc_epoch             0.9805653691291809
  val_boundary_loss_epoch               0.0
val_cross_entropy_loss_epoch     0.3543056845664978
       val_loss_epoch            0.3543056845664978
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────


Validation: |          | 0/? [00:00<?, ?it/s]

────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      Validate metric               DataLoader 0
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
       val_acc_epoch             0.9402782320976257
  val_boundary_loss_epoch               0.0
val_cross_entropy_loss_epoch     0.4108952283859253
       val_loss_epoch            0.4108952283859253
────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
